<a href="https://colab.research.google.com/github/yuehaoshi/Solar-Energy-Appliation/blob/main/Visualization%20Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install chart_studio
!pip install matplotlib opencv-python 

  Using cached https://github.com/matplotlib/basemap/archive/master.zip
ERROR: File "setup.py" not found for legacy project https://github.com/matplotlib/basemap/archive/master.zip.


In [ ]:
import numpy as np
import math
import requests
import plotly
import pandas as pd
from chart_studio.plotly import plotly as py
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import cv2
from matplotlib.patches import Patch
from matplotlib.lines import Line2D
from google.colab.patches import cv2_imshow
from mpl_toolkits.basemap import Basemap as Basemap
from matplotlib.colors import rgb2hex, Normalize
from matplotlib.patches import Polygon
from matplotlib.cm import ScalarMappable
from matplotlib.colorbar import ColorbarBase
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: ignored

##Part 1: Earth-Solar System

1.1 Helper Functions for Spheres and Orbit

In [ ]:
def spheres(size, clr, dist=0): 
    
    # Set up 100 points. First, do angles
    theta = np.linspace(0,2*np.pi,100)
    phi = np.linspace(0,np.pi,100)
    
    # Set up coordinates for points on the sphere
    x0 = dist + size * np.outer(np.cos(theta),np.sin(phi))
    y0 = size * np.outer(np.sin(theta),np.sin(phi))
    z0 = size * np.outer(np.ones(100),np.cos(phi))
    
    # Set up trace
    if clr == 0:
      trace = go.Surface(x=x0, y=y0, z=z0, surfacecolor=-x0**2 - y0**2 - z0**2)
    else:
      trace = go.Surface(x=x0, y=y0, z=z0, colorscale=[[0,clr], [1,clr]])
    trace.update(showscale=False)

    return trace

In [ ]:
def orbits(dist, offset=0, clr='white', wdth=2): 
    
    # Initialize empty lists for eac set of coordinates
    xcrd=[]
    ycrd=[]
    zcrd=[]
    
    # Calculate coordinates
    for i in range(0,361):
        xcrd=xcrd+[(round(np.cos(math.radians(i)),5)) * dist + offset]
        ycrd=ycrd+[(round(np.sin(math.radians(i)),5)) * dist]
        zcrd=zcrd+[0]
    
    trace = go.Scatter3d(x=xcrd, y=ycrd, z=zcrd, marker=dict(size=0.1), line=dict(color=clr,width=wdth))
    return trace

In [ ]:
def annot(xcrd, zcrd, txt, xancr='center'):
    strng=dict(showarrow=False, x=xcrd, 
               y=0, z=zcrd, text=txt, 
               xanchor=xancr, font=dict(color='white',size=12))
    return strng

1.2 Model of earth-solar system

In [ ]:
diameter_km = [200000, 127560]
diameter = [((i / 12756) * 2) for i in diameter_km]
distance_from_sun = [0, 149.6]

# Create spheres for the Sun and planets
trace0=spheres(diameter[0], '#ffff00', distance_from_sun[0]) # Sun
trace1=spheres(diameter[1], '#325bff', distance_from_sun[1]) # Earth

# Set up orbit traces
trace2 = orbits(distance_from_sun[1]) # Earth

layout=go.Layout(title = 'Solar System', showlegend=False, margin=dict(l=0, r=0, t=0, b=0),
                  #paper_bgcolor = 'black',
                  scene = dict(xaxis=dict(title='Distance from the Sun', 
                                          titlefont_color='black', 
                                          range=[-300,300], 
                                          backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'),
                               yaxis=dict(title='Distance from the Sun',
                                          titlefont_color='black',
                                          range=[-300,300],
                                          backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'
                                          ),
                               zaxis=dict(title='', 
                                          range=[-300,300],
                                          backgroundcolor='black',
                                          color='white', 
                                          gridcolor='black'
                                         ),
                               annotations=[
                                   annot(distance_from_sun[0], 40, 'Sun', xancr='left'),
                                   annot(distance_from_sun[1], 9, 'Earth'),
                                   ]
                               ))

fig = go.Figure(data = [trace0, trace1, trace2],
                layout = layout)

fig.show()

1.3 Visualization about sunlight effect to earth

In [ ]:
diameter_km = [200000, 127560]
diameter = [((i / 12756) * 2) for i in diameter_km]
distance_from_sun = [0, 149.6]

# Create spheres for the Sun and planets
trace0=spheres(diameter[0], '#ffff00', distance_from_sun[0]) # Sun
trace1=spheres(diameter[1], 0, distance_from_sun[1]) # Earth

# Set up orbit traces
trace2 = orbits(distance_from_sun[1]) # Earth

layout=go.Layout(title = 'Solar System', showlegend=False, margin=dict(l=0, r=0, t=0, b=0),
                  #paper_bgcolor = 'black',
                  scene = dict(xaxis=dict(title='Distance from the Sun', 
                                          titlefont_color='black', 
                                          range=[-300,300], 
                                          backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'),
                               yaxis=dict(title='Distance from the Sun',
                                          titlefont_color='black',
                                          range=[-300,300],
                                          backgroundcolor='black',
                                          color='black',
                                          gridcolor='black'
                                          ),
                               zaxis=dict(title='', 
                                          range=[-300,300],
                                          backgroundcolor='black',
                                          color='white', 
                                          gridcolor='black'
                                         ),
                               annotations=[
                                   annot(distance_from_sun[0], 40, 'Sun', xancr='left'),
                                   annot(distance_from_sun[1], 9, 'Earth'),
                                   ]
                               ))

fig = go.Figure(data = [trace0, trace1, trace2],
                layout = layout)

fig.show()

#Part 2

States boundary of US

In [ ]:
# Lambert Conformal map of lower 48 states.
'''m = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)'''
m = Basemap(llcrnrlon=-121,llcrnrlat=20,urcrnrlon=-62,urcrnrlat=51,
    projection='lcc',lat_1=32,lat_2=45,lon_0=-95)
# draw state boundaries.
# data from U.S Census Bureau
# http://www.census.gov/geo/www/cob/st2000.html
shp_info = m.readshapefile('/content/drive/MyDrive/Courses/CS519/cb_2018_us_state_20m/cb_2018_us_state_20m','states',drawbounds=True)

NameError: ignored

PSH map for each state

In [ ]:
#Adapted from https://github.com/matplotlib/basemap/blob/master/examples/fillstates.py
m = Basemap(llcrnrlon=-121,llcrnrlat=20,urcrnrlon=-62,urcrnrlat=51,
    projection='lcc',lat_1=32,lat_2=45,lon_0=-95)
# draw state boundaries.
# data from U.S Census Bureau
# http://www.census.gov/geo/www/cob/st2000.html
shp_info = m.readshapefile('/content/drive/MyDrive/Courses/CS519/cb_2018_us_state_20m/cb_2018_us_state_20m','states',drawbounds=True, color="white", linewidth=0)
shp_info
PeakSunHour = {
'Alabama':3.75,
'Alaska': 2.5,
'Arizona':7.5,
'Arkansas':3.75,
'California':6.25,
'Colorado':5.75,
'Connecticut':3,
'Delaware':4,
'Florida':4,
'Georgia':4.25,
'Hawaii':4.5,
'Idaho':4.25,
'Illinois':3.5,
'Indiana':3.25,
'Iowa':4,
'Kansas':4.75,
'Kentucky':3.5,
'Louisiana':4.25,
'Maine':3.25,
'Maryland':3.5,
'Massachusetts':3,
'Michigan':3,
'Minnesota':4,
'Mississippi':4.25,
'Missouri':4.25,
'Montana':4.5,
'Nebraska':4.75,
'Nevada':6.75,
'New Hampshire':3.25,
'New Jersey':3.75,
'New Mexico':6.5,
'New York':3.25,
'North Carolina':4.25,
'North Dakota':4,
'Ohio':3,
'Oklahoma':5,
'Oregon':4,
'Pennsylvania':3,
'Rhode Island':3.5,
'South Carolina':4.25,
'South Dakota':4.75,
'Tennessee':4,
'Texas':5.25,
'Utah':6.5,
'Vermont':3.25,
'Virginia':3.75,
'Washington':3.75,
'West Virginia':3,
'Wisconsin':3.5,
'Wyoming':5.75
}
# choose a color for each state based on population density.
colors={}
statenames=[]
cmap = plt.cm.YlOrRd # use 'YlOrRd' colormap
#cmap = plt.cm.summer
vmin = 2.4; vmax = 7.6 # set range.
#vmin = min(popdensity.values()); vmax = max(popdensity.values())
norm = Normalize(vmin=vmin, vmax=vmax)
mapper = ScalarMappable(norm=norm, cmap=cmap)
for shapedict in m.states_info:
    statename = shapedict['NAME']
    # skip DC and Puerto Rico.
    if statename in PeakSunHour:
        hour = PeakSunHour[statename]
        # calling colormap with value between 0 and 1 returns
        # rgba value.  Invert color range (hot colors are high
        # population), take sqrt root to spread out colors more.
        colors[statename] = mapper.to_rgba(hour)
    statenames.append(statename)
# cycle through state names, color each one.
ax = plt.gca() # get current axes instance
fig = plt.gcf()
fig.set_size_inches(15,10)

for nshape,seg in enumerate(m.states):
    # skip DC and Puerto Rico.
    if statenames[nshape] not in ['Puerto Rico', 'District of Columbia']:
    # Offset Alaska and Hawaii to the lower-left corner. 
        if statenames[nshape] == 'Alaska':
        # Alaska is too big. Scale it down to 35% first, then transate it. 
            seg = list(map(lambda xy: (0.35*xy[0] + 1100000, 0.35*xy[1]-1300000), seg))
        if statenames[nshape] == 'Hawaii':
            seg = list(map(lambda xy: (xy[0] + 5200000, xy[1]-1400000), seg))

        color = rgb2hex(colors[statenames[nshape]]) 
        poly = Polygon(seg,facecolor=color,edgecolor=color, linewidth=.5)
        ax.add_patch(poly)
plt.title('Filling each State by Average Peak Sun Hours (PSH)')

# construct custom colorbar
cax = fig.add_axes([0.27, 0, 0.5, 0.05]) # posititon
cb = ColorbarBase(cax,cmap=cmap,norm=norm, orientation='horizontal')
#cb.ax.invert_xaxis()
cb.ax.set_xlabel('PSH')
plt.show()

NameError: ignored

In [ ]:
img = cv2.imread('/content/drive/MyDrive/Courses/CS519/Screen Shot 2021-11-28 at 22.01.26.png')
x = []
y = []
z = []
for i in range(img.shape[0]):
  curr = []
  y.append(i)
  for j in range(img.shape[1]):
    rgb = 65536 * img[i][j][0] + 256 * img[i][j][1] + img[i][j][2];
    curr.append(rgb)
    if i == 0:
      x.append(j)
  z.append(curr)
#print("x", x)
#print("y", y)
#print("shapez", np.shape(z))
#print("z", z)
#z = np.ma.array(z, mask=(z == 16777215))
y.reverse()

cmap=plt.cm.summer
cmap.set_over('white')
cs = plt.contourf(x,y,z,5, cmap=cmap, extend="max")
plt.axis('off')

#Part3

Annual kWh Produced per DC kW for various Tilts / Azimuths

In [ ]:
z = []
x = [i for i in range(90, 275, 5)]
y = [j for j in range(0, 95, 5)]
for j in y:
  cur_energy = []
  for i in x:
    intensity = 3.5 * (math.cos(math.radians(26.39)) * math.sin(math.radians(j)) * math.cos(math.radians(i - 185.43)) + math.sin(math.radians(26.39)) * math.cos(math.radians(j)))
    intensity = 0 if intensity < 0 else intensity
    cur_energy.append(intensity)
  z.append(cur_energy)

trace1 = go.Contour(
    z=z,
    x=x,
    y=y,
    colorscale='Portland',
)

trace2 = go.Scatter(
    x=[90, 135, 180, 225, 270],
    y=[3.5] * len(x),
    mode='text',
    text=['East', 'Southeast', 'South', 'Southwest', 'West'],
    textposition='bottom center',
    textfont=dict(
        family='Arial',
        size=20,
        color='black'
    )
)

layout = go.Layout(
    title='Daily kWh Produced per DC kW for various Tilts / Azimuths in Chicago at 11/28/21',
    titlefont=dict(
        family='Arial',
        size=20,
    ),
    
    width=800,
    height=600,
    xaxis=dict(
        title='Direction Faced by Panels, degrees',
        titlefont=dict(
            family='Arial',
            size=22,
        ),
        autorange = True,
        ticks = 'outside',
    ),
    yaxis=dict(
        title='Tilt of Panels from Horizontal, degrees',
        titlefont=dict(
            family='Arial',
            size=22,
        ),
    )
)

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig)

Represent contour in 3D axis

In [ ]:
fig = go.Figure(data=[go.Surface(x = x, y = y, z = z)])
fig.update_traces(contours_z=dict(show=True, usecolormap=True,
                                  highlightcolor="limegreen", project_z=True),
                  opacity=0.8,
                  )
fig.update_layout(title='Annual kWh Produced per DC kW for various Tilts / Azimuths', 
                  scene = dict(
                    xaxis = dict(
                         backgroundcolor="rgb(200, 200, 230)",
                         gridcolor="white",
                         showbackground=True,
                         zerolinecolor="white",),
                    yaxis = dict(
                        backgroundcolor="rgb(230, 200,230)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white"),
                    zaxis = dict(
                        backgroundcolor="rgb(230, 230,200)",
                        gridcolor="white",
                        showbackground=True,
                        zerolinecolor="white",),
                    xaxis_title='Azimuth',
                    yaxis_title='Tilt',
                    zaxis_title='Energy'),
                  autosize=False,
                  scene_camera_eye=dict(x=1.87, y=0.88, z=-0.64),
                  width=600, height=600,
                  margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()